In [7]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [15]:
from llmxml import parse_xml, generate_prompt_template
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

class Movie(BaseModel):
    title: str = Field(..., description="The title of the movie", examples=[])
    director: str = Field(..., description="The director of the movie", examples=["The Wachowskis", "The Wachowskis", "The Wachowskis"])


class MovieList(BaseModel):
    movies: list[Movie] = Field(
        ..., description="A list of movies that match the query", examples=[Movie(title="The Matrix", director="The Wachowskis"), Movie(title="The Matrix Reloaded", director="The Wachowskis"), Movie(title="The Matrix Revolutions", director="The Wachowskis")]
    )

class Response(BaseModel):
    movies: list[Movie] = Field(
        ..., description="A list of movies that match the query"
    )

class ResponseObject(BaseModel):
    response: Response = Field(
        ..., description="The response object that contains the movies"
    )

xml: str = """
<response>
    <movies>
        <movie>
            <title>The Matrix</title>
            <director>The Wachowskis</director>
        </movie>
    </movies>
</response>
"""

result: ResponseObject = parse_xml(ResponseObject, xml)
print(result)

response=Response(movies=[Movie(title='The Matrix', director='The Wachowskis')])


In [16]:
import json
movie = MovieList(movies=[Movie(title="The Matrix", director="The Wachowskis"), Movie(title="The Matrix Reloaded", director="The Wachowskis"), Movie(title="The Matrix Revolutions", director="The Wachowskis")], examples=[Movie(title="The Matrix", director="The Wachowskis"), Movie(title="The Matrix Reloaded", director="The Wachowskis"), Movie(title="The Matrix Revolutions", director="The Wachowskis")])
print(json.dumps(movie.model_json_schema(), indent=2))

{
  "$defs": {
    "Movie": {
      "properties": {
        "title": {
          "description": "The title of the movie",
          "examples": [],
          "title": "Title",
          "type": "string"
        },
        "director": {
          "description": "The director of the movie",
          "examples": [
            "The Wachowskis",
            "The Wachowskis",
            "The Wachowskis"
          ],
          "title": "Director",
          "type": "string"
        }
      },
      "required": [
        "title",
        "director"
      ],
      "title": "Movie",
      "type": "object"
    }
  },
  "properties": {
    "movies": {
      "description": "A list of movies that match the query",
      "examples": [
        {
          "director": "The Wachowskis",
          "title": "The Matrix"
        },
        {
          "director": "The Wachowskis",
          "title": "The Matrix Reloaded"
        },
        {
          "director": "The Wachowskis",
          "title": "The

In [17]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI

# Define your desired output structure
class ExtractUser(BaseModel):
    name: str = Field(..., description="The name of the user")
    age: int = Field(..., description="The age of the user")

# Patch the OpenAI client
client_instructor = instructor.from_openai(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

# Extract structured data from natural language
res = client_instructor.chat.completions.create(
    model="gpt-4o-mini",
    response_model=ExtractUser,
    messages=[{"role": "user", "content": "give a random name and age in xml format"}],
)
print(res)

name='John Doe' age=30


In [1]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import os
import llmxml
from dotenv import load_dotenv
load_dotenv()

# Define your desired output structure
class ExtractUser(BaseModel):
    name: str = Field(..., description="The name of the user")
    age: int = Field(..., description="The age of the user")

# Patch the OpenAI client
client_instructor = llmxml.from_openai(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

# Extract structured data from natural language
res = client_instructor.chat.completions.create(
    model="gpt-4o-mini",
    response_model=ExtractUser,
    messages=[{"role": "user", "content": "give a random name and age in xml format"}],
)
print(res)

name='John Doe' age=28


In [3]:
from pydantic import BaseModel
import anthropic
import instructor

# Patching the Anthropics client with the instructor for enhanced capabilities
client = instructor.patch(anthropic.Anthropic())


class Properties(BaseModel):
    key: str
    value: str


class User(BaseModel):
    name: str
    age: int
    properties: list[Properties]


user = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    max_retries=0,
    messages=[
        {
            "role": "user",
            "content": "Create a user for a model with a name, age, and properties.",
        }
    ],
    response_model=User,
)

print(user.model_dump_json(indent=2))


AttributeError: module 'instructor' has no attribute 'from_anthropic'

In [ ]:
from pydantic import BaseModel
import anthropic
import instructor

# Patching the Anthropics client with the instructor for enhanced capabilities
client = instructor.from_anthropic(anthropic.Anthropic())


class Properties(BaseModel):
    key: str
    value: str


class User(BaseModel):
    name: str
    age: int
    properties: list[Properties]


user = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    max_retries=0,
    messages=[
        {
            "role": "user",
            "content": "Create a user for a model with a name, age, and properties.",
        }
    ],
    response_model=User,
)

print(user.model_dump_json(indent=2))


In [18]:
client_openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
prompt: str = generate_prompt_template(
    model=ExtractUser, include_instructions=True  # Default is true
)
print(prompt)

res = client_openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt},
        {"role": "user", "content": f"give a random name and age."},
    ],
)

print(res.choices[0].message.content)
result = parse_xml(ExtractUser, res.choices[0].message.content)
print(result)

<response_instructions>
You are to provide your output in the following xml-like format EXACTLY as described in the schema provided.

Each field in the schema has a description and a type enclosed in square brackets, denoting that they are metadata.

Format instructions:
<field_name>
[object_type]
[description]
</field_name>


Basic example:

<EXAMPLE>
<EXAMPLE_SCHEMA>
<thinking>
[type: str]
[Chain of thought]
</thinking>
<actions>
# Option 1: CommandAction
<command_action>
<action_type>
[type: Literal["command"]]
[The type of action to perform]
</action_type>
<command>
[type: str]
[The command to run]
</command>
</command_action>

OR

<action>
# Option 2: CreateAction
<create_action>
<action_type>
[type: Literal["create"]]
[The type of action to perform]
</action_type>
<new_file_path>
[type: str]
[The path to the new file to create]
</new_file_path>
<file_contents>
[type: str]
[The contents of the new file to create]
</file_contents>
</create_action>

OR

# Option 3: EditAction
<edit_

In [ ]:
# Synchronous usage
from openai import OpenAI
from llmxml import from_openai
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., description="User's name")
    age: int = Field(..., description="User's age")

client = from_openai(OpenAI())

response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=User,
    messages=[{"role": "user", "content": "Generate a random user"}]
)
print(response.name, response.age)

# Asynchronous usage
# from openai import AsyncOpenAI
# import asyncio

# async def main():
#     async_client = from_openai(AsyncOpenAI())
    
#     response = await async_client.chat.completions.create(
#         model="gpt-4",
#         response_model=User,
#         messages=[{"role": "user", "content": "Generate a random user"}]
#     )
#     print(response.name, response.age)

# asyncio.run(main())